<a href="https://colab.research.google.com/github/Klnishant/ANNImplementation/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [2]:
class SentimentAnalysis(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
    super(SentimentAnalysis, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size,2)

  def forward(self,x):
    emmbed = self.embedding(x)
    output, _ = self.lstm(emmbed)
    output = self.fc(output[:, -1, :])
    return output

In [3]:
vocab_size = 10000
embedding_dim = 100
hidden_size =128
num_layers = 2
sentiment_model = SentimentAnalysis(vocab_size,embedding_dim,hidden_size,num_layers)

In [4]:
train_data = torch.randint(0,vocab_size,(128,50))
train_labels = torch.randint(0,2,(128,))

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(sentiment_model.parameters(),lr=0.01)

In [6]:
num_epochs = 10
for epoch in range(num_epochs):
  optimizer.zero_grad()
  outputs = sentiment_model(train_data)
  loss = criterion(outputs,train_labels)
  loss.backward()
  optimizer.step()
  print(f"Epoch: {epoch+1}/{num_epochs} loss: {loss}")

Epoch: 1/10 loss: 0.6917958855628967
Epoch: 2/10 loss: 0.6508482098579407
Epoch: 3/10 loss: 0.4306378960609436
Epoch: 4/10 loss: 1.1808702945709229
Epoch: 5/10 loss: 0.2537427246570587
Epoch: 6/10 loss: 0.17313073575496674
Epoch: 7/10 loss: 0.1726241409778595
Epoch: 8/10 loss: 0.1492934674024582
Epoch: 9/10 loss: 0.10371389985084534
Epoch: 10/10 loss: 0.06096195802092552


In [7]:
class NERLSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes):
    super(NERLSTM, self).__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size,num_classes)

  def forward(self,x):
    embbed = self.embedding(x)
    output,_ = self.lstm(embbed)
    output = self.fc(output)
    return output

In [8]:
num_classes =10
ner_model = NERLSTM(vocab_size,embedding_dim,hidden_size,num_layers,num_classes)

In [9]:
criterion_ner = nn.CrossEntropyLoss()
optimizer_ner = torch.optim.Adam(ner_model.parameters(),lr=0.01)

In [10]:
train_data_ner = torch.randint(0, vocab_size, (128, 10))  # Example data with batch size 128 and sequence length 30
train_labels_ner = torch.randint(0, num_classes, (128, 10))  # Example NER labels


In [12]:
for epoch in range(num_epochs):
  optimizer_ner.zero_grad()
  outputs = ner_model(train_data_ner)
  loss = criterion_ner(outputs,train_labels_ner)
  loss.backward()
  optimizer_ner.step()
  print(f"Epoch: {epoch+1}/{num_epochs} loss: {loss.item():.4f}")

Epoch: 1/10 loss: 0.9012
Epoch: 2/10 loss: 0.7354
Epoch: 3/10 loss: 0.5868
Epoch: 4/10 loss: 0.4490
Epoch: 5/10 loss: 0.3360
Epoch: 6/10 loss: 0.2419
Epoch: 7/10 loss: 0.1686
Epoch: 8/10 loss: 0.1156
Epoch: 9/10 loss: 0.0788
Epoch: 10/10 loss: 0.0539


In [13]:
class LanguageGeneration(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
    super(LanguageGeneration, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, vocab_size)

  def forward(self,x):
    embbed = self.embedding(x)
    output,_ = self.lstm(embbed)
    output = self.fc(output)
    return output

In [14]:
language_model = LanguageGeneration(vocab_size,embedding_dim,hidden_size,num_layers)

In [15]:
train_data_lang = torch.randint(0,vocab_size,(128,30))

In [16]:
criterion_lang = nn.CrossEntropyLoss()
optimizer_lang = torch.optim.Adam(language_model.parameters(),lr=0.01)

In [18]:
for epoch in range(num_epochs):
  optimizer_lang.zero_grad()
  outputs = language_model(train_data_lang)
  loss = criterion_lang(outputs.view(-1,vocab_size),train_data_lang.view(-1))
  loss.backward()
  optimizer_lang.step()
  print(f"Epoch: {epoch+1}/{num_epochs} loss: {loss.item():.4f}")

Epoch: 1/10 loss: 7.7088
Epoch: 2/10 loss: 7.5929
Epoch: 3/10 loss: 7.4566
Epoch: 4/10 loss: 7.3373
Epoch: 5/10 loss: 7.1752
Epoch: 6/10 loss: 7.0041
Epoch: 7/10 loss: 6.8227
Epoch: 8/10 loss: 6.6358
Epoch: 9/10 loss: 6.4318
Epoch: 10/10 loss: 6.2197


In [19]:
def generate_text(model, start_token, max_length=50):
  model.eval()
  with torch.no_grad():
    current_token = start_token
    generated_text = [start_token]

    for _ in range(max_length):
      input_tensor = torch.tensor([[current_token]])
      output = model(input_tensor)
      next_token_probs = output[:,-1,:]
      next_token = torch.argmax(next_token_probs).item()
      generated_text.append(next_token)
      current_token = next_token

    return generated_text

In [20]:
start_token = torch.randint(0, vocab_size, (1,1))
generated_text = generate_text(language_model,start_token,max_length=100)
print("Generated Text")
print(generated_text)

Generated Text
[tensor([[1391]]), 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643, 5643]
